## Import necessary headers

In [188]:
from pynq import Overlay
from pynq import MMIO
from pynq import allocate
import random
import string
import numpy as np
import sys
import hashlib
import timeit
print("User Current Version:-", sys.version)

User Current Version:- 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]


## Generate input

In [220]:
num = 10
def gen_input(num):
    '''
    Generate num random inputs
    @Output rand_inputs: random generated ndarray of uint32
    @Output frags: number of fragments
    '''
    
    origin_inputs = []
    rand_inputs = []
    frags = []
    for i in range(num):
        N = random.randint(1, 1000)
        rand_bytes = [random.randint(0, sys.maxsize) for i in range(N)]
        
        rand_bytes.append(0x70000000)
        zeros = 14 - N % 16
        zeros = zeros if zeros >= 0 else zeros + 16
        rand_bytes += [0 for i in range(zeros)]
        rand_bytes += rand_bytes[0:2]
        rand_bytes = np.array(rand_bytes)
        
    
        rand_inputs.append(rand_bytes)
        frags.append((len(rand_bytes)) // 16)
        origin_inputs.append(rand_bytes)
    
    return rand_inputs, origin_inputs, frags
inputs, origins, frags = gen_input(num)

## Software SHA-256

In [221]:
k = np.array([
   0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
   0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
   0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
   0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
   0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
   0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
   0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
   0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2], dtype=np.uint32)

def rightRotate(n, d):
    return (n >> d) | (n << (32 - d)) & 0xFFFFFFFF

def sha_256(sha_input, frags):
    start = timeit.default_timer()
    acc = np.array([0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
                    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19], dtype=np.uint32)
    for iteration in range(frags):
        w = []
        for i in range(16):
            w.append(sha_input[i + iteration * 16])
        for i in range(16, 64):
            s0 = rightRotate(w[i - 15], 7) ^ rightRotate(w[i - 15], 18) ^ (w[i-15] >> 3)
            s1 = rightRotate(w[i - 2], 17) ^ rightRotate(w[i - 2], 19) ^ (w[i-2] >> 10)
            w.append((w[i - 16] + s0 + w[i - 7] + s1) & 0xffffffff)
        tmp = np.copy(acc)
        for i in range(64):
            s0 = rightRotate(tmp[0], 2) ^ rightRotate(tmp[0], 13) ^ rightRotate(tmp[0], 22)
            maj = (tmp[0] & tmp[1]) ^ (tmp[0] & tmp[2]) ^ (tmp[1] & tmp[2])
            t2 = s0 + maj
            s1 = rightRotate(tmp[4], 6) ^ rightRotate(tmp[4], 11) ^ rightRotate(tmp[4], 25)
            ch = (tmp[4] & tmp[5]) ^ (~tmp[4] & tmp[6])
            t1 = tmp[7] + s1 + ch + k[i] + w[i]
            
            tmp[7], tmp[6], tmp[5], tmp[4], tmp[3], tmp[2], tmp[1], tmp[0] = \
            tmp[6], tmp[5], tmp[4], (tmp[3] + t1) & 0xffffffff, tmp[2], tmp[1], tmp[0], (t1 + t2) & 0xffffffff
        acc += tmp
    return acc, timeit.default_timer() - start


print(frags)
total_time = 0
for i in range(len(frags)):
    hash_val, interval = sha_256(inputs[i], frags[i])
    output = ''
    for i in range(8):
        output += str(hex(hash_val[i]))[2:].zfill(8)
    print(output)
    total_time += interval
print("total time: ", total_time)

[27, 12, 57, 13, 11, 27, 43, 33, 9, 48]
cd389354b46b184d234b590e7930f43425d8e02e03b29fc9537f280b6319b352
536f91d757b17e03319d0eaeca94d559e10185c527b57523864080e422958c0a
59d314ae944edb275e16e39fb664eca75640109efd6003e5858390f04ccb06b8
075776c9fa87e466fde5e8e5d5e01725920825a18ea0735ce6221a855112f6d4
79c28fafb5a09350c3afd60682ba15fdebd6784af8b386ed887a101c7c02cb3e
6bad11b7039bed8288d229e709a436d7a8f1d6303cf74b59d814910ebbede216
e2853c9a90b83f90b263e1db1cdc54cc71e0aab24ccf253d695218d5a84a8042
9e28a77592fbc0df3fa9b26af305650c029f805ebc3e949c25dec01c923c11e9
a3de2e9cab24ca788f38d9acbc4855bd3d57379faf0917843258f331ea9b89a6
570628e3295f885dcd3818fba13880077a8043ed2d5aea4e74a1ed4feb8af2fc
total time:  21.885697254998377


In [222]:
start = timeit.default_timer()

for i in range(len(origins)):
    m = hashlib.sha256()
    for j in range(len(origins[i])):
        m.update(inputs[i][j])
    print(m.hexdigest())
print("total time: ", timeit.default_timer() - start)

972135aab4940c8b0bf8cc67ee19428699d76a93c9ff998b157b329ec2c3b88c
c0405a78808eb7373eae08ada74013379615600133fdae1edf9ddb189f560cfa
444c1eb8265881e971bceab598235b31a79f0ef539c4db081e3c1c41b795c362
352d917713b710443adbe6f0fc74c1d62fa69b88ddb1b98c515515c9d731074d
da8c2c3911cd4c4b86f16400bae4cfe0a3b31da2e153d87e2a8db7c7b33583b1
a89f417ad12c20d657bed19d9e49887021a28fff1035ee330cfdc1ad78b47111
298d4f095ec2f950d52dcad1fcc95495baad429dbe8804f2ea0c6c4aa548cfe3
2c52a7bd59374c044f9b7930890db596b0f8e9064d0fac6f47bfffd6c3d4d0bc
ba30347560229b5061a031515b4537281e363f9fcc34445305e021e8b13e2ee8
648d808097eff9df04d0f61f4028c134792bd2f2dfc79f3a3c4e4f856f2156d1
total time:  0.09331553900119616


## PL SHA-256

In [ ]:
BRAM0_ADDR = Lab_Design.ip_dict['axi_bram_ctrl_0']['phys_addr']
BRAM0_ADDR_range = Lab_Design.ip_dict['axi_bram_ctrl_0']['addr_range']
BRAM0_MMIO = MMIO(BRAM0_ADDR, BRAM0_ADDR_range)
BRAM1_ADDR = Lab_Design.ip_dict['axi_bram_ctrl_1']['phys_addr']
BRAM1_ADDR_range = Lab_Design.ip_dict['axi_bram_ctrl_1']['addr_range']
BRAM1_MMIO = MMIO(BRAM0_ADDR, BRAM0_ADDR_range)
BRAM2_ADDR = Lab_Design.ip_dict['axi_bram_ctrl_2']['phys_addr']
BRAM2_ADDR_range = Lab_Design.ip_dict['axi_bram_ctrl_2']['addr_range']
BRAM2_MMIO = MMIO(BRAM0_ADDR, BRAM0_ADDR_range)
BRAM3_ADDR = Lab_Design.ip_dict['axi_bram_ctrl_3']['phys_addr']
BRAM3_ADDR_range = Lab_Design.ip_dict['axi_bram_ctrl_3']['addr_range']
BRAM3_MMIO = MMIO(BRAM0_ADDR, BRAM0_ADDR_range)

GPIO0_ADDR = Lab_Design.ip_dict['axi_gpio_0']['phys_addr']           # size_ins
GPIO0_ADDR_range = Lab_Design.ip_dict['axi_gpio_0']['addr_range']
GPIO1_ADDR = Lab_Design.ip_dict['axi_gpio_1']['phys_addr']           # start / clear
GPIO1_ADDR_range = Lab_Design.ip_dict['axi_gpio_1']['addr_range']
GPIO2_ADDR = Lab_Design.ip_dict['axi_gpio_2']['phys_addr']           # valid
GPIO2_ADDR_range = Lab_Design.ip_dict['axi_gpio_2']['addr_range']
GPIO0_MMIO = MMIO(GPIO0_ADDR, GPIO0_ADDR_range)
GPIO1_MMIO = MMIO(GPIO1_ADDR, GPIO1_ADDR_range)
GPIO2_MMIO = MMIO(GPIO2_ADDR, GPIO2_ADDR_range)

bram_addrs = [BRAM0_ADDR, BRAM1_ADDR, BRAM2_ADDR, BRAM3_ADDR]
bram_ranges = [BRAM0_ADDR_range, BRAM1_ADDR_range, BRAM2_ADDR_range, BRAM3_ADDR_range]
bram_mmio = [BRAM0_MMIO, BRAM1_MMIO, BRAM2_MMIO, BRAM3_MMIO]

In [223]:
start_sigs = [0, 0, 0, 0]

q = []

def send_size_sig(n, fragments):
    GPIO0_MMIO.write(0, n << 6 + fragments)

def send_start_sig():
    val = 0
    for i in start_sigs:
        val << 1
        val += start_sigs
    GPIO1_MMIO.write(0, start_sigs)
    
def parse_valid_sig(valid):
    has_valid = 0
    for i in range(4):
        x = valid & 0x1
        if (x == 1):
            q.append(i)  # i is ready for next input
            start_sigs[i] = 0
            has_valid = 1
        valid >> 1
    send_start_sig()
    return has_valid

def get_result(n):
    for i in range(0, 32, 4):
        x = bram_mmio[n].read(i)
        output = ''
        output += hex(x)[2:].zfill(8)
    print(output)

cur_input = 0

for i in range(4):
    for j in range(len(inputs[cur_input])): 
        bram_mmio[i].write(j * 4, int(inputs[cur_input][j]))
    start_sigs[i] = 1
    send_size_sig(i, frags[cur_inpts])
    send_start_sig(i)
    cur_input += 1
    if cur_input > num:
        break;

while (cur_input < num):
    while (!parse_valid_sig()):
        continue
    for i in q:
        get_result(i)
        for j in range(len(inputs[cur_input])): 
            bram_mmio[i].write(j * 4, int(inputs[cur_input][j]))
        start_sigs[i] = 1
        send_size_sig(i, frags[cur_inpts])
        send_start_sig(i)
        cur_input += 1


SyntaxError: unexpected EOF while parsing (<ipython-input-223-e851406935a2>, line 6)